# Test generative models

In [4]:
import torch

In [34]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")

# testing gpt2
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [107]:
# complete pipeline

model_id = 'bigscience/bloom-560m'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

text = "Question: how to print 'hello world' in python?\n Answer: "
inputs = tokenizer(
    text, 
    truncation = True, 
    max_length = 512, 
    return_tensors = 'pt'
)
output = model.generate(**inputs)
print(tokenizer.decode(output[0]))

/Users/daniel/miniforge3/envs/torch_new/lib/python3.9/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Question: how to print 'hello world' in python?
 Answer:  #!/usr/bin/env python
import
